In [23]:
#Written by GC -- 02/24

#This notebook contains code to generate ign dists for each NTS sheet based of the national fire database (NFDB). The overall approach here is to
#create a circular buffer around an NTS sheet, calc the ratio of fire per year to fuel pixels for this buffered area, and then multiple the the number of
#fuel pix per NTS sheet to get a spatially smoothed ign number that is based on real-world fire data.  
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
#import contextily as cx
import numpy as np
import matplotlib.cm as cm
import rasterio as rio
import rioxarray as rxr
from ztp_funcs import ztp_dist
import os

In [14]:
#Read in nts sheet shapefile
df_nts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\nts_snrc\\nts_snrc_250k.shp")
df_nts=df_nts.to_crs(epsg=6931) #Convert to equal-area projection

In [15]:
#Read in NFDB data and select fires larger than 1 ha between 2010-2020. This is done to 1) implicity 
#include the impact of fire suppression and 2) remove fires that too small to be simulated by BP3+
df_pnts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\NFDB_point\\NFDB_point_20220901.shp")
df_pnts=df_pnts[df_pnts['YEAR'].between(2010,2020)]
df_pnts=df_pnts[df_pnts['SIZE_HA'].ge(6.25)]

#Now count number of fires in each NTS sheet

#Check which NTS sheet a fire is in
df_pnts=df_pnts.to_crs(df_nts.crs)
fire_loc_ls=[]
for index, row in df_pnts.iterrows():
    inside=df_nts.geometry.contains(row.geometry)
    try:
        fire_loc_ls.append(df_nts[inside]['NTS_SNRC'].values[0])
    except:
        pass

#Aggregate and average to get mean annual fires per NTS sheet
df_numfires=pd.Series(fire_loc_ls).value_counts().to_frame()
df_numfires.index.name='NTS_SNRC'
df_numfires_large=df_nts.merge(df_numfires, on='NTS_SNRC')
df_numfires_large=df_numfires_large.rename(columns={'count':'lg_count'})
df_numfires_large['lg_count']=df_numfires_large['lg_count']/11

In [16]:
#Calculate the number of fuel pixels per NTS sheet.
fbp = rxr.open_rasterio("C:\\Users\\GiovanniCorti\\Downloads\\Canadian_Forest_FBP_Fuel_Types_v20191114\\fuel_layer\\FBP_FuelLayer.tif")
dt=df_nts.to_crs(fbp.rio.crs)

nts_list=[]
fuel_list=[]
for index, row in dt.iterrows():
    polys = gpd.GeoSeries((row.geometry))
    nts_list.append(row['NTS_SNRC'])
    try:
        clip_fbp=fbp.rio.clip(polys, from_disk=True)
        fuel=np.where((clip_fbp.values > 100) & (clip_fbp.values <150), 0, 1)
        fuel_list.append(np.sum(fuel))
    except:
        fuel_list.append(0)

dict = {'NTS_SNRC': nts_list, 'f_pix': fuel_list} 
df_nts_fpix=pd.DataFrame(dict)


In [17]:
#Buffer NTS sheet centroids to create a 150km radius circle around each sheet. This area, 
# which overlaps with circles from adjacent NTS sheets will be used for spatial smoothing
df_ignb_L=df_nts.copy()
df_ignb_L=df_ignb_L.to_crs(epsg='6931')
df_ignb_L['geometry']=df_ignb_L['geometry'].centroid.buffer(250000)

df_ignb_H=df_nts.copy()
df_ignb_H=df_ignb_H.to_crs(epsg='6931')
df_ignb_H['geometry']=df_ignb_H['geometry'].centroid.buffer(100000)


In [18]:
#Read in NFDB data and select fires larger than 1 ha between 2010-2020. This is done to 1) implicity 
#include the impact of fire suppression and 2) remove fires that too small to be simulated by BP3+
df_pnts=gpd.read_file("C:\\Users\\GiovanniCorti\\Downloads\\NFDB_point\\NFDB_point_20220901.shp")
df_pnts=df_pnts[df_pnts['YEAR'].between(2010,2020)]
df_pnts=df_pnts[df_pnts['SIZE_HA'].ge(10)]
#df_pnts=df_pnts.to_crs(epsg='4326')
df_pnts=df_pnts.to_crs(epsg='6931')
df_ignb_H['count']=0
df_ignb_L['count']=0

df_pnts_H=df_pnts[df_pnts["CAUSE"]=="H"]
df_pnts_L=df_pnts[df_pnts["CAUSE"]=="L"]

#Check which fires fall within an NTS circle
#H_fire_loc_ls=[]
for index, row in df_ignb_H.iterrows():
    inside=df_pnts_H['geometry'].within(row.geometry)
    num_igns=len(inside[inside==True])
    try:
        df_ignb_H.at[index,'count']=num_igns
    except:
        print('fail')
        pass
    
    
#Check which fires fall within an NTS circle
#L_fire_loc_ls=[]
for index, row in df_ignb_L.iterrows():
    inside=df_pnts_L['geometry'].within(row.geometry)
    num_igns=len(inside[inside==True])
    try:
        df_ignb_L.at[index,'count']=num_igns
    except:
        print('fail')
        pass
    


In [9]:
#Calc number of fuel pixels in each NTS circle (i.e., buffered sheet) at human ignition scale (100 km radius)
fbp = rxr.open_rasterio("C:\\Users\\GiovanniCorti\\Downloads\\Canadian_Forest_FBP_Fuel_Types_v20191114\\fuel_layer\\FBP_FuelLayer.tif")
dt=df_ignb_H.to_crs(fbp.rio.crs)

nts_list=[]
fuel_list=[]
for index, row in dt.iterrows():
    polys = gpd.GeoSeries((row.geometry))
    nts_list.append(row['NTS_SNRC'])
    try:
        clip_fbp=fbp.rio.clip(polys, from_disk=True)
        fuel=np.where((clip_fbp.values < 100) | (clip_fbp.values >150), 1, 0)
        fuel_list.append(np.sum(fuel))
    except:
        fuel_list.append(0)

dict = {'NTS_SNRC': nts_list, 'f_pix_buff': fuel_list} 
dt=pd.DataFrame(dict)

#Calc fire to fuel pix ratio for NTS circle and multiply by num fuel pix per sheet. 
df_buff=df_ignb_H.merge(dt, on='NTS_SNRC')
df_buff['ratio']=df_buff['count']/df_buff['f_pix_buff']
df_buff=df_buff.merge(df_nts_fpix, on='NTS_SNRC',how='left')
df_buff['ign_num_H']=(df_buff['f_pix']*df_buff['ratio'])/11
df_ign_num=df_nts.merge(df_buff[['NTS_SNRC','ign_num_H']], on='NTS_SNRC', how='left')
df_ign_num_H=df_ign_num.to_crs(epsg=3857)


In [10]:
#Calc number of fuel pixels in each NTS circle (i.e., buffered sheet) at lightning ignition scale (250 km radius)
fbp = rxr.open_rasterio("C:\\Users\\GiovanniCorti\\Downloads\\Canadian_Forest_FBP_Fuel_Types_v20191114\\fuel_layer\\FBP_FuelLayer.tif")
dt=df_ignb_L.to_crs(fbp.rio.crs)

nts_list=[]
fuel_list=[]
for index, row in dt.iterrows():
    polys = gpd.GeoSeries((row.geometry))
    nts_list.append(row['NTS_SNRC'])
    try:
        clip_fbp=fbp.rio.clip(polys, from_disk=True)
        fuel=np.where((clip_fbp.values < 100) | (clip_fbp.values >150), 1, 0)
        fuel_list.append(np.sum(fuel))
    except:
        fuel_list.append(0)

dict = {'NTS_SNRC': nts_list, 'f_pix_buff': fuel_list} 
dt=pd.DataFrame(dict)

#Calc fire to fuel pix ratio for NTS circle and multiply by num fuel pix per sheet. 
df_buff=df_ignb_L.merge(dt, on='NTS_SNRC')
df_buff['ratio']=df_buff['count']/df_buff['f_pix_buff']
df_buff=df_buff.merge(df_nts_fpix, on='NTS_SNRC',how='left')
df_buff['ign_num_L']=(df_buff['f_pix']*df_buff['ratio'])/11
df_ign_num=df_nts.merge(df_buff[['NTS_SNRC','ign_num_L']], on='NTS_SNRC', how='left')
df_ign_num_L=df_ign_num.to_crs(epsg=3857)

In [11]:
df_ign_num=df_ign_num_L.merge(df_ign_num_H[['NTS_SNRC','ign_num_H']],on="NTS_SNRC" )
df_ign_num['ign_num']=df_ign_num['ign_num_H']+df_ign_num['ign_num_L']

#df_ign_num[df_ign_num['NTS_SNRC']=='082L']
df_ign_num[df_ign_num['ign_num']>0].sample(30)

,NTS_SNRC,NAME_ENG,NOM_FRA,SRID,SHAPE_AREA,SHAPE_LEN,geometry,ign_num_L,ign_num_H,ign_num
789,053M,KNEE LAKE,KNEE LAKE,6,2.0,6.0,"POLYGON ((-10684816.948 7361867.861, -10686672...",1.957973,0.065681,2.023654
580,073M,WINEFRED LAKE,WINEFRED LAKE,6,2.0,6.0,"POLYGON ((-12465929.304 7361867.529, -12467784...",1.930630,1.168783,3.099413
1139,105P,SEKWI MOUNTAIN,SEKWI MOUNTAIN,6,2.0,6.0,"POLYGON ((-14469681.030 9100252.115, -14471536...",0.889022,0.000000,0.889022
316,033O,LAC MONTROCHAND,LAC MONTROCHAND,6,2.0,6.0,"POLYGON ((-8458426.343 7361868.139, -8460281.6...",0.257845,0.000000,0.257845
941,052I,ARMSTRONG,ARMSTRONG,6,2.0,6.0,"POLYGON ((-10016899.667 6446277.389, -10018754...",1.449089,0.221437,1.670526
461,083L,WAPITI,WAPITI,6,2.0,6.0,"POLYGON ((-13356485.383 7170157.479, -13358340...",1.273742,0.304437,1.578180
905,065F,ENNADAI,ENNADAI,6,2.0,6.0,"POLYGON ((-11352734.361 8625825.228, -11354589...",0.358486,0.000000,0.358486
194,023J,SCHEFFERVILLE,SCHEFFERVILLE,6,2.0,6.0,"POLYGON ((-7567870.061 7170158.307, -7569725.3...",0.399463,0.060269,0.459732
303,030L,BUFFALO,BUFFALO,6,2.0,6.0,"POLYGON ((-8682920.578 5160980.692, -8684775.9...",0.042318,0.000000,0.042318
75,012K,MUSQUARO,MUSQUARO,6,2.0,6.0,"POLYGON ((-6899952.839 6446277.605, -6901808.1...",0.427004,0.068345,0.495348


In [12]:
#Save to shape file
df_ign_num.to_file("C:/Users/GiovanniCorti/Documents/Wildfire/ign_test_2.shp")

In [36]:

#Create ign dist .csvs for each NTS sheet
for index, row in df_ign_num.iterrows():
    if np.isnan(row["ign_num"]):
        pass
    
    elif row["ign_num"]<1:
        tdict = {'ign_per_it': [0,1], 'pct': [100-np.round(row["ign_num"]*100,2),np.round(row["ign_num"]*100,2)]}
        dt=pd.DataFrame(tdict)
        if os.path.isdir("C:\\Users\\GiovanniCorti\\Desktop\\BP3Inputs\\"+row['NTS_SNRC']):
            dt.to_csv("C:\\Users\\GiovanniCorti\\Desktop\\BP3Inputs\\"+row['NTS_SNRC']+"\\ign_dist_"+row['NTS_SNRC']+'.csv',index=False)
    elif row["ign_num"]>=1:
        ztp_df=ztp_dist('Igns', row["ign_num"])
        tdict = {'ign_per_it': ztp_df["Value"], 'pct': np.round(ztp_df["RelativeFrequency"]*100,2)} 
        dt=pd.DataFrame(tdict)
        #dt.to_csv("Y:client-data\\demo_projects\\climate85\\Working_data\\SED_dist_v2\\sed_dist_"+row['NTS_SNRC']+'.csv',index=False)
        if os.path.isdir("C:\\Users\\GiovanniCorti\\Desktop\\BP3Inputs\\"+row['NTS_SNRC']):
            dt.to_csv("C:\\Users\\GiovanniCorti\\Desktop\\BP3Inputs\\"+row['NTS_SNRC']+"\\ign_dist_"+row['NTS_SNRC']+'.csv',index=False)

,NTS_SNRC,NAME_ENG,NOM_FRA,SRID,SHAPE_AREA,SHAPE_LEN,geometry,ign_num_L,ign_num_H,ign_num
1,001C,None,None,6,2.0,6.0,"POLYGON ((-6009396.588 5465443.612, -6011251.9...",0.000000,0.000000,0.000000
2,001D,None,None,6,2.0,6.0,"POLYGON ((-6232035.645 5465443.611, -6233890.9...",0.000000,0.000000,0.000000
3,001E,None,None,6,2.0,6.0,"POLYGON ((-6232035.646 5621522.967, -6233890.9...",0.000000,0.000000,0.000000
4,001F,None,None,6,2.0,6.0,"POLYGON ((-6009396.588 5621522.968, -6011251.9...",0.000000,0.000000,0.000000
5,002E,BOTWOOD,BOTWOOD,6,2.0,6.0,"POLYGON ((-6232035.649 6274863.105, -6233890.9...",0.023307,0.072898,0.096206
6,002F,WESLEYVILLE,WESLEYVILLE,6,2.0,6.0,"POLYGON ((-6009396.587 6274863.105, -6011251.9...",0.011749,0.000000,0.011749
7,002G,None,None,6,2.0,6.0,"POLYGON ((-5786757.526 6274863.103, -5788612.8...",0.000000,0.000000,0.000000
8,002H,None,None,6,2.0,6.0,"POLYGON ((-5564118.464 6274863.100, -5565973.7...",0.000000,0.000000,0.000000
9,001O,None,None,6,2.0,6.0,"POLYGON ((-5786757.528 5942075.664, -5788612.8...",0.000000,0.000000,0.000000
